In [1]:
import mcstasscript as ms
import make_powder_instrument
import quizlib

%matplotlib widget

import numpy as np
import mcstastox
import scipp as sc

from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import elastic

def read_event_union(data, source_name="Source", sample_name="sample_position"):
    with mcstastox.Read(data[0].original_data_location) as file:
        source_position, source_rotation = file.get_component_placement(source_name)
        sample_position, sample_rotation = file.get_component_placement(sample_name)
        
        detector_comps = file.get_components_with_ids()
        global_positions_all = None    
    
        for detector in detector_comps:
            mon = ms.name_search(detector, data)
    
            y = mon.get_data_column("y")
            positions = np.column_stack((np.zeros_like(y), y, np.zeros_like(y)))
            global_positions = file.transform(positions, detector)
    
            if global_positions_all is None:
                global_positions_all = global_positions
                event_weights = mon.get_data_column("p")
                event_times = mon.get_data_column("t")
            else:
                global_positions_all = np.vstack((global_positions_all, global_positions))
                event_weights = np.concatenate((event_weights, mon.get_data_column("p")))
                event_times = np.concatenate((event_times, mon.get_data_column("t")))
    
        events = sc.DataArray(
                data=sc.array(dims=['events'], unit=sc.units.counts, values=event_weights),
                coords={
                    'position' : sc.vectors(dims=['events'], values=global_positions_all, unit='m'),
                    't': sc.array(dims=['events'], unit='s', values=event_times),
                    'source_position': sc.vector(source_position, unit='m'),
                    'sample_position': sc.vector(sample_position, unit='m'),                
                })

        return events

def read_event_classic(data, source_name="Source", sample_name="sample_position"):
    with mcstastox.Read(data[0].original_data_location) as file:
        events = file.export_scipp_simple(source_name, sample_name)

        detector_name_1 = "Banana_large"
        detector_name_2 = "Banana_small"
        variables_1 = file.get_component_variables(detector_name_1)
        variables_2 = file.get_component_variables(detector_name_2)
        if "L" in variables_1 and "L" in variables_2:
            raw_event_data_1 = events.get_event_data(variables=["id", "t", "L"], component_name=detector_name_1)
            raw_event_data_2 = events.get_event_data(variables=["id", "t", "L"], component_name=detector_name_2)
            full_L = np.concatenate((raw_event_data_1["L"], raw_event_data_2["L"]))
            
    events.coords["x"] = sc.array(dims=["events"], values=events.coords["position"].fields.x.values, unit="m")
    events.coords["y"] = sc.array(dims=["events"], values=events.coords["position"].fields.y.values, unit="m")
    events.coords["z"] = sc.array(dims=["events"], values=events.coords["position"].fields.z.values, unit="m")

    # Get ground truth simulated wavelength
    if "L" in variables_1 and "L" in variables_2:
        events.coords["sim_wavelength"] = sc.array(dims=["events"], values=raw_event_data["L"], unit="Å")

    return events


def analyze(data, union_detectors=False):

    if union_detectors:
        events = read_event_union(data)
    else:
        events = read_event_classic(data)
    
    # McStas provides absolute time, not time of flight
    events.coords["tof"] = events.coords["t"]# - sc.to_unit(sc.scalar(2.86/2, unit="ms"), "s")
    
    graph = {**beamline(scatter=True), **elastic("tof")}
    events = events.transform_coords("dspacing", graph=graph)

    return events

def run(par_dict=None, settings_dict=None, union_detectors=False):

    if par_dict is None:
        par_dict = {}

    if settings_dict is None:
        settings_dict = {}

    if "NeXus" not in settings_dict:
        settings_dict["NeXus"] = True

    instrument = make_powder_instrument.make(union_detectors=False)
    instrument.set_parameters(**par_dict)
    instrument.settings(**settings_dict)
    data = instrument.backengine()

    events = analyze(data)

    return events

def plot_with_dspacings(events, filename, d_min=0):
    d_hist = events.hist(dspacing=sc.linspace("dspacing", 0.6, 3.0, num=800, unit="Å", endpoint=False))
    max_val = max(d_hist.values)

    laz = np.loadtxt(filename)
    d_spacings = laz[:, 5]
    intensity = laz[:, 11]

    intensity_normalized = intensity/intensity.max()

    fig =  d_hist.plot()

    for d_space, intensity_norm in zip(d_spacings, intensity_normalized):
        if d_space > d_min:
            fig.ax.plot([d_space, d_space], [0, max_val], "-k", alpha=intensity_norm)
    
    return fig

def run_and_plot(par_dict=None, settings_dict=None, union_detectors=False):
    events = run(par_dict, settings_dict, union_detectors)

    if "reflections" in par_dict:
        filename = par_dict["reflections"].strip(('"'))
    else:
        filename = "Na2Ca3Al2F14.laz"
    
    return plot_with_dspacings(events, filename, d_min=0.5)

In [2]:
instrument = make_powder_instrument.make(union_detectors=False)
instrument.show_parameters()

       l_min                      = 0.5                 // Minimum simulated wavelength [AA]
       l_max                      = 4.0                 // Maximum simulated wavelength [AA]
int    n_pulses                   = 1                   // Number of simulated pulses
       guide_curve_deg            = 1.0                 // 
       chopper_wavelength_center  = 2.5                 // Center of wavelength band [AA]
double frequency_multiplier       = 1                   // [1] Chopper frequency as multiple of source frequency
       detector_height            = 2.5                 // 
string reflections                = "Na2Ca3Al2F14.laz"  // 
       sample_radius              = 0.01                // 
       sample_height              = 0.05                // 


### Quick run for testing

In [3]:
pars = dict(reflections='"Fe.laz"', frequency_multiplier=3, guide_curve_deg=0, detector_height=1.5)
settings = dict(suppress_output=True, ncount=5E7, mpi=10)

fig = run_and_plot(pars, settings)
fig

InteractiveFigure(children=(HBar(), HBar(children=(VBar(children=(Toolbar(children=(ButtonTool(icon='home', la…

### Run and work with scipp object directly

In [4]:
settings["ncount"]=1E9
events = run(pars, settings)

In [5]:
graph = {**beamline(scatter=True), **elastic("tof")}
events = events.transform_coords("wavelength", graph=graph)

events

<scipp.DataArray>
Dimensions: Sizes[events:331524, ]
Coordinates:
  L1                        float64              [m]  ()  160.639
  L2                        float64              [m]  (events)  [3.5002, 3.55758, ..., 3.5002, 3.53379]
  Ltotal                    float64              [m]  (events)  [164.139, 164.197, ..., 164.139, 164.173]
* dspacing                  float64             [Å]  (events)  [2.12476, 5.98341, ..., 15.3217, 4.64496]
  incident_beam             vector3              [m]  ()  (0, 0, 160.639)
  position                  vector3              [m]  (events)  [(2.6112, 0.0375, 158.308), (1.99499, 0.6375, 163.515), ..., (-0.831901, -0.0375, 164.039), (-1.73676, -0.4875, 163.678)]
  sample_position           vector3              [m]  ()  (0, 0, 160.639)
  scattered_beam            vector3              [m]  (events)  [(2.6112, 0.0375, -2.33059), (1.99499, 0.6375, 2.87576), ..., (-0.831901, -0.0375, 3.3997), (-1.73676, -0.4875, 3.0387)]
  source_position           vector3              [m]  ()  (0, 0, 0)
* t                         float64              [s]  (events)  [0.160914, 0.153754, ..., 0.152341, 0.102038]
  tof                       float64              [s]  (events)  [0.160914, 0.153754, ..., 0.152341, 0.102038]
  two_theta                 float64            [rad]  (events)  [2.29942, 0.629457, ..., 0.240217, 0.535727]
* wavelength                float64             [Å]  (events)  [3.8783, 3.70443, ..., 3.67168, 2.45878]
* x                         float64              [m]  (events)  [2.6112, 1.99499, ..., -0.831901, -1.73676]
* y                         float64              [m]  (events)  [0.0375, 0.6375, ..., -0.0375, -0.4875]
* z                         float64              [m]  (events)  [158.308, 163.515, ..., 164.039, 163.678]
Data:
                            float64         [counts]  (events)  [0.000161021, 0.00040973, ..., 0.00057903, 8.16039e-05]

In [6]:
import plopp as pp

pp.scatter3d(events[0::3], pos='position', size=0.02, cbar=True, norm="linear")

Figure(children=(HBar(children=(HTML(value=''),)), HBox(children=(VBar(children=(Toolbar(children=(ButtonTool(…

In [7]:
#events.coords["y"] = events.coords["position"].fields.y
#del events.coords["y"]
events

<scipp.DataArray>
Dimensions: Sizes[events:331524, ]
Coordinates:
  L1                        float64              [m]  ()  160.639
  L2                        float64              [m]  (events)  [3.5002, 3.55758, ..., 3.5002, 3.53379]
  Ltotal                    float64              [m]  (events)  [164.139, 164.197, ..., 164.139, 164.173]
* dspacing                  float64             [Å]  (events)  [2.12476, 5.98341, ..., 15.3217, 4.64496]
  incident_beam             vector3              [m]  ()  (0, 0, 160.639)
  position                  vector3              [m]  (events)  [(2.6112, 0.0375, 158.308), (1.99499, 0.6375, 163.515), ..., (-0.831901, -0.0375, 164.039), (-1.73676, -0.4875, 163.678)]
  sample_position           vector3              [m]  ()  (0, 0, 160.639)
  scattered_beam            vector3              [m]  (events)  [(2.6112, 0.0375, -2.33059), (1.99499, 0.6375, 2.87576), ..., (-0.831901, -0.0375, 3.3997), (-1.73676, -0.4875, 3.0387)]
  source_position           vector3              [m]  ()  (0, 0, 0)
* t                         float64              [s]  (events)  [0.160914, 0.153754, ..., 0.152341, 0.102038]
  tof                       float64              [s]  (events)  [0.160914, 0.153754, ..., 0.152341, 0.102038]
  two_theta                 float64            [rad]  (events)  [2.29942, 0.629457, ..., 0.240217, 0.535727]
* wavelength                float64             [Å]  (events)  [3.8783, 3.70443, ..., 3.67168, 2.45878]
* x                         float64              [m]  (events)  [2.6112, 1.99499, ..., -0.831901, -1.73676]
* y                         float64              [m]  (events)  [0.0375, 0.6375, ..., -0.0375, -0.4875]
* z                         float64              [m]  (events)  [158.308, 163.515, ..., 164.039, 163.678]
Data:
                            float64         [counts]  (events)  [0.000161021, 0.00040973, ..., 0.00057903, 8.16039e-05]

In [8]:
events_binned = events.bin(wavelength=120, dspacing=800)
pp.slicer(events_binned.hist(), keep=["wavelength", "dspacing"], norm="log", vmax=1E6, vmin=1E-3)

InteractiveFigure(children=(HBar(), HBar(children=(VBar(children=(Toolbar(children=(ButtonTool(icon='home', la…

In [9]:
events.hist(t=200).plot()

InteractiveFigure(children=(HBar(), HBar(children=(VBar(children=(Toolbar(children=(ButtonTool(icon='home', la…

### Run instrument directly without the help functions

In [10]:
instrument = make_powder_instrument.make(union_detectors=False)

In [ ]:
instrument.show_parameters()

In [ ]:
instrument.set_parameters(frequency_multiplier=8, reflections='"Fe.laz"')
instrument.settings(ncount=1e7, mpi=10, suppress_output=True, NeXus=True, output_path="first_run")

In [ ]:
data = instrument.backengine()

In [ ]:
ms.make_sub_plot(data, log=True, orders_of_mag=5)

In [ ]:
ms.make_sub_plot([data[0].make_2d("th", "y"), data[1].make_2d("th", "y")], log=True, orders_of_mag=5)

### Visualize instrument (opens in new window)

In [ ]:
#instrument.show_instrument(format="window")